# Module 4 - Integration Amazon Q index with Amazon Nova Tool use

### Use Amazon Q index to search relevant content and Tool use (function calling) with Amazon Nova models


This module demonstrates how to leverage the Tool use functionality with Amazon Nova to analyze support tickets using the retrieved content from an Amazon Q index. In the earlier modules, we used the Amazon Q Business `SearchRelevantContent` API on the cross-app index to perform a semantic and keyword (Hybrid) based search. In this section, we will use these relevant information chunks to invoke tool functions and process the input parameters provided by the model. We will cover the complete process of defining and integrating tools when working with Amazon Nova models.

Before we proceed further, let's install some pre-requisites first. Run the following code-block.

<div class="alert alert-block alert-warning"> <b>IMPORTANT:</b> Please make sure that you select <b>"Python 3 (ipykernel)"</b> kernel for this notebook from the top right corner, if one is not selected already. </div>

## Step 1

In [ ]:
!python -m pip install boto3


#### Pre-requisites

In order to be able to call Amazon Q Business's data APIs, we will need to acquire credentials that are tagged to a specific user id (in this case, an email address). We have pre-deployed a mechanism that will help generate this credentials with a helper function. First we will need to acquire some necessary details that will help us generate the identity-aware SigV4 AWS credentials. Specifically, we will require the following details.

- `issuer`: the issuer URL
- `client_id`: a client_id for the OIDC client
- `client_secret`: a client_secret for the OIDC client
- `role_arn`: the IAM role to assume, this is the role that has Amazon Q Business permissions
- `region`: `us-east-1` this is the current region where our Amazon Q Business Application is setup 
- `email`: your email address (can be a fictional email address of the format user@email.com)

To obtain the values, execute the next code cell which will read a JSON file from an S3 bucket.

## Step 2

In [ ]:
import boto3
import json
import sagemaker

s3_client = boto3.client('s3')
role = sagemaker.get_execution_role()
account_id = role.split(':')[4]
bucket_name = f'amazon-q-tvm-{account_id}'
file_key = 'tvm_values.json'

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
creds = json.loads(response['Body'].read().decode('utf-8'))["TVMOidcIssuerStack"]
creds["IssuerUrlOutput"] = creds["IssuerUrlOutput"].rstrip('/')

We will use the JSON data above to initialize our helper function `TVMClient` in the next code block. Go ahead and enter your email address in the `email_address` variable if you wish to, else you can keep `jon_doe@email.com`. Once done, execute the code block.

## Step 3

In [ ]:
from utils.tvm_client import TVMClient

email_address="john_doe@email.com"

token_client = TVMClient(
        issuer=creds["IssuerUrlOutput"],
        client_id=creds["QbizTVMClientID"],
        client_secret=creds["QbizTVMClientSecret"],
        role_arn=creds["QBizAssumeRoleARN"],
        region="us-east-1"
)
    
# Get Sigv4 credentials using TVM
credentials = token_client.get_sigv4_credentials(email=email_address)

## Step 4

_Use SigV4 credentials to initialize Amazon Q Business Client_: We will then initialize an Amazon Q Business Boto3 (Python) client with the SigV4 credentials obtained using the helper function to make calls to Amazon Q Business APIs (in this case the `SearchRelevantContent` API).

In [ ]:
import boto3

qbiz = boto3.client("qbusiness", **credentials)
    

## Calling `SearchRelevantContent` API
---

In the previous code block, we initialized an Amazon Q Business client with the required credentials. We are now ready to make the call to the `SearchRelevantContent` API and analyze it's response. To call the API, we require the Amazon Q Business Application id (`applicationId`) and the retriever ID (`retrieverId`). 

Execute the next code block under "Step 5" which will fetch the application ID and retriever ID respectively.

## Step 5

In [ ]:
Q_BIZ_APP_ID = ""
Q_RETRIEVER_ID = ""

import boto3
client = boto3.client('qbusiness')
response_app = client.list_applications()

for r in response_app["applications"]:
    if 'aim333-module-2' in r['displayName']:
        Q_BIZ_APP_ID=r['applicationId']

if Q_BIZ_APP_ID:
    response_ret = client.list_retrievers(applicationId=Q_BIZ_APP_ID)
    Q_RETRIEVER_ID = response_ret['retrievers'][0]['retrieverId']

print(f"Application ID is: {Q_BIZ_APP_ID}\nRetriever ID is: {Q_RETRIEVER_ID}")

<div class="alert alert-block alert-info"> 
    <b><i>(Optional)</i> View Application ID and Retriever ID in the Amazon Q Business console:</b> 
    <p>
    You can also get the values for application ID and retriever ID from the Amazon Q Business console under the application named `aim333-module-2`. Follow these steps to view the values (this is informational only, if you have executed the code block above you may move on to the next step) -</p>

<ol>
    <li> Navigate to the Amazon Q Business console.</li>
    <li> Click on "Applications" from the menu on the left.</li>
    <li> Click on the application named `aim333-module-2`.</li>
    <li> In the following screen, scroll down to the "Application Settings" section.</li>
    <li> The application ID is found under the label "Application ID".</li>    
</ol>
<p>Next, to get the retriever ID</p>
<ol>
    <li> In the same screen, click on the next tab labeled "Index".</li>
    <li> The retriever ID is found under the label "Retriever ID".</li>
</ol>
</div>

Next we will run the `SearchRelevantContent` API.

In [ ]:
import boto3
import pprint
qbiz = boto3.client("qbusiness", region_name="us-east-1", **credentials)


search_params = {  'applicationId': Q_BIZ_APP_ID, 
    'contentSource': {
        'retriever': { 
            'retrieverId': Q_RETRIEVER_ID 
            }
    }, 
    'queryText': 'What are the reasons for keyboard failures?', 
    'maxResults': 5
}

def call_search_relevant_content(search_params):
    try:
        search_response = qbiz.search_relevant_content(**search_params)
        pprint.pprint(search_response['relevantContent'])
        return search_response['relevantContent']
    except Exception as e:
        print(e)

relevant_content = call_search_relevant_content(search_params)

In [ ]:
full_context = ""

for chunks in relevant_content:
    full_context = full_context + chunks['content'] + "\n"

print(full_context)

## Tool use with Amazon Nova

Now that we have the full context and any relevant information related to the query, we will then orchestrate the Tool use with Amazon Q index integration following these high-level steps:

1) **User Query** - User submits a query that may require both knowledge retrieval and tool execution
2) **Tool Selection** - Amazon Nova analyzes both the user query and retrieved context to determine if a tool is necessary
3) **Return Results** - Amazon Nova incorporates both the retrieved context and tool output into a comprehensive final response


<div class="alert alert-block alert-warning">
<b>NOTE:</b> You must have completed the pre-requisites in the beginning of the workshop to enable <b>Amazon Nova Lite</b> model access in order to proceed with this section of the hands-on. Also note that we are specifically accessing Bedrock in `us-west-2` region. </div>

## Step 6

Here is an example of how you will define a tool:

In [ ]:
# Define Nova ToolUse configuration
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "AnalyzeContent",
                "description": "Analyzes retrieved content and provides insights.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "context": {"type": "string"},
                            "query": {"type": "string"}
                        },
                        "required": ["context", "query"]
                    }
                }
            }
        }
    ]
}

Now that we've defined our tool, let's implement the code that calls Nova and handles any tool invocations. 

We will use the [Amazon Bedrock `Converse` API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html) to let the Nova model use a tool in a conversation. The Converse API example shows how to synchronously use the tool in our use case.

Nova models follow this specific sequence when using tools:

1. **Initial Response:** Model acknowledges your request and outlines its plan
2. **Tool Execution:** Model uses the required tool(s)
3. **Follow-up Response:** Model analyzes and explains tool results

In [ ]:
query="What are the reasons for keyboard failures?"

# Create the initial message from the user input.
initial_response = bedrock_client.converse(
    modelId="us.amazon.nova-lite-v1:0",
    messages=[
        {"role": "user", "content": [{"text":f"Given the full context:{full_context}\n\nAnalyze the following content for: {query}"}]}
    ],
    toolConfig=tool_config,
    inferenceConfig={"maxTokens": 1000, "temperature": 0.7}
)

print(initial_response)

Now, We will invoke the tool. When a tool is invoked, its output is seamlessly passed back to Nova as a `toolResult`, allowing the model to incorporate this information into its final response.

In [ ]:
tool_use = None
for content_block in initial_response['output']['message']['content']:
    if 'toolUse' in content_block:
        tool_use = content_block['toolUse']
        break

# Tool use requested. Call the tool and send the result to the model.
if tool_use:
    print("Tool use requested with ID:", tool_use['toolUseId'])
    
    # Provide tool results back to the model
    follow_up_response = bedrock_client.converse(
        modelId="us.amazon.nova-lite-v1:0",
        messages=[
            {"role": "user", "content": [{"text": f"Analyze the following content for: {query}."}]},
            {"role": "assistant", "content": initial_response['output']['message']['content']},
            {
                "role": "user",
                "content": [
                    {
                        "toolResult": {
                            "toolUseId": tool_use['toolUseId'],
                            "content": [{"text": json.dumps({
                                "context": full_context,
                                "query": query
                            })}]
                        }
                    }
                ]
            }
        ],
        toolConfig=tool_config,
        inferenceConfig={"maxTokens": 1000, "temperature": 0.7}
    )

    final_analysis = follow_up_response['output']['message']['content'][0]['text']
else:
    # If no tool is triggered, use the initial_response
    final_analysis = initial_response['output']['message']['content'][0]['text']

print("Nova Analysis:\n", final_analysis)

## Sending Analysis Results via Amazon SNS

We'll set up an Amazon SNS topic to deliver Nova's analysis to users. This ensures immediate notification of results through email.

## Step 7

In [ ]:
#Set Up SNS Topic and Subscription
sns_client = boto3.client('sns')
response = sns_client.create_topic(Name="ticketanalyzer")
topic_arn = response['TopicArn']
print(f"SNS Topic created successfully: {topic_arn}")

In [ ]:
email = "<YOUR EMAIL ADDRESS>"
response = sns_client.subscribe(
            TopicArn=topic_arn,
            Protocol='email',
            Endpoint=email
        )
subscription_arn = response['SubscriptionArn']
print(f"Subscription initiated: {subscription_arn}")

In [ ]:
# Send the notification
email_subject = "Ticket analysis by AnyCompany AI"

response = sns_client.publish(
    TopicArn=topic_arn,
    Message=final_analysis,
    Subject=email_subject,
)
message_id = response['MessageId']
print(f"Formatted email sent successfully! Message ID: {message_id}")

# Conclusion
---

In this module, we've explored the powerful integration of Amazon Q Business's cross-app index with Amazon Nova's tool use capabilities. We demonstrated how to harness the `SearchRelevantContent` API's results to create sophisticated tools with Amazon Nova, combining function calling with intelligent analysis based on user queries. This notebook template provides a comprehensive foundation that you can build upon—whether by adding custom tools, enhancing error handling, or adapting the analysis patterns to your specific needs. Thank you for participating in this workshop! We hope these insights help you build more powerful and context-aware applications using Amazon Q index and Amazon Nova.